In [29]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [30]:
from skforecast.preprocessing import reshape_series_long_to_dict
from skforecast.preprocessing import reshape_exog_long_to_dict

In [31]:
import pandas as pd
import numpy as np
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series

,datetime,series_id,value
0,2023-01-01,series_1,59
1,2023-01-02,series_1,50
2,2023-01-03,series_1,27
3,2023-01-04,series_1,75
4,2023-01-05,series_1,13
...,...,...,...
2494995,2036-09-04,series_499,11
2494996,2036-09-05,series_499,43
2494997,2036-09-06,series_499,69
2494998,2036-09-07,series_499,27


In [32]:
series_multiindex = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
series_multiindex

value
series_id  datetime         
series_1   2023-01-01     59
           2023-01-02     50
           2023-01-03     27
           2023-01-04     75
           2023-01-05     13
...                      ...
series_499 2036-09-04     11
           2036-09-05     43
           2036-09-06     69
           2036-09-07     27
           2036-09-08     65

[2495000 rows x 1 columns]

In [33]:
reshape_series_long_to_dict(
    data = series,
    freq = "D",
    # series_id = 'series_id',
    # index  = 'datetime',
    # values = 'value',
    # suppress_warnings = False
)

ValueError: Arguments `series_id`, `index` and `values` cannot be `None` when input data does not have MultiIndex. Please provide a value for each.

In [34]:
exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog_multiindex = exog.set_index(["series_id", exog.index])
exog = exog.reset_index()
exog

,datetime,exog_1,exog_2,series_id
0,2000-01-01,100.0,1000.0,series_1
1,2000-01-02,101.0,1001.0,series_1
2,2000-01-03,102.0,1002.0,series_1
3,2000-01-04,103.0,1003.0,series_1
4,2000-01-05,104.0,1004.0,series_1
5,2000-01-06,105.0,1005.0,series_1
6,2000-01-07,106.0,1006.0,series_1
7,2000-01-08,107.0,1007.0,series_1
8,2000-01-09,108.0,1008.0,series_1
9,2000-01-10,109.0,1009.0,series_1


In [39]:
reshape_exog_long_to_dict(
    data = exog,
    freq = "D",
    series_id = 'series_id',
    index  = 'datetime',
    suppress_warnings = False
)

{'series_1':             exog_1  exog_2
 2000-01-01   100.0  1000.0
 2000-01-02   101.0  1001.0
 2000-01-03   102.0  1002.0
 2000-01-04   103.0  1003.0
 2000-01-05   104.0  1004.0
 2000-01-06   105.0  1005.0
 2000-01-07   106.0  1006.0
 2000-01-08   107.0  1007.0
 2000-01-09   108.0  1008.0
 2000-01-10   109.0  1009.0,
 'series_2':             exog_1  exog_2
 2000-01-01   100.0  1000.0
 2000-01-02   101.0  1001.0
 2000-01-03   102.0  1002.0
 2000-01-04   103.0  1003.0
 2000-01-05   104.0  1004.0
 2000-01-06   105.0  1005.0
 2000-01-07   106.0  1006.0
 2000-01-08   107.0  1007.0
 2000-01-09   108.0  1008.0
 2000-01-10   109.0  1009.0}